In [80]:
import numpy as np
import cv2
import logging
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm, metrics

In [76]:


# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# img = cv2.imread('WIN_20201209_17_25_54_Pro.jpg')
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# faces = face_cascade.detectMultiScale(gray, 1.3, 5)
# for (x,y,w,h) in faces:
#     img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#     roi_gray = gray[y:y+h, x:x+w]
#     roi_color = img[y:y+h, x:x+w]

# cv2.imshow('img',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [36]:
# various configurations for plotting and logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [54]:
import os
from skimage import img_as_ubyte, io, color

In [141]:
def load_training_data(path):
    if not os.path.exists(path):
        logger.info('Please download training/testing data to CW_Dataset directory')
        return None
    else:
        images = list()
        labels = list()
        
    file_list = np.genfromtxt(path + '/labels/list_label_train.txt', dtype=str)    
    logger.info(f"Loading {len(file_list)} images")
    for f_name, label in file_list:
        img = io.imread(os.path.join(path, 'train', f_name.split('.jpg')[0]+'_aligned.jpg'))
        if img is not None:
            images.append(img)
            labels.append(label)
        else:
            logger.info(f'Error loading image at {f_name}')
    logger.info(f'Successfully loaded {len(images)} images')
    return images, labels
            
path = '../../CW_Dataset'

In [142]:
X, y = load_training_data(path)

INFO:__main__:Loading 12271 images
INFO:__main__:Successfully loaded 12271 images


In [180]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, stratify=y)

In [181]:
class SIFT():
    def __init__(self, k):
        self.k = k
        self.sift = cv2.SIFT_create()
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        des_arr = list()
        y_list = list()
        logger.info('Beginning SIFT transformations')
        for i in range(len(X)):
            img = img_as_ubyte(color.rgb2gray(X[i]))
            kp, des = self.sift.detectAndCompute(img, None)
            
            if des is not None:
                des_arr.append(des)
                y_list.append(y[i])       
        return des_arr, y_list
    
    def fit_transform(self, X, y, **fit_params):
        return self.fit(X).transform(X, y=y)
    
    
class MiniKMeans():
    def __init__(self):
        self.num_clusters = None
        self.batch_size = None
        self.model = None
        
    def fit(self, X, y, *args, **kwargs):
        X = np.vstack(X)
        self.num_clusters = len(np.unique(y)) * 10
        self.batch_size = X.shape[0] // 4
        self.model = MiniBatchKMeans(n_clusters=self.num_clusters, batch_size=self.batch_size).fit(X)
        return self
        
    def transform(self, X):  
        hist_list = list()
        for des in X:
            hist = np.zeros(self.num_clusters)
            idx =  self.model.predict(des)
            for j in idx:
                hist[j] = hist[j] + (1 / len(des))
            
            hist_list.append(hist)
        return np.vstack(hist_list)

In [182]:
class EmotionRecSVC():
    def __init__(self, kernel):
        self.kernel = kernel
        self.model = None
        
    def fit_transform(self, X, y):
        X, y = SIFT(10).fit_transform(X, y)
        X = MiniKMeans().fit(X, y).transform(X)
        
        self.model = svm.SVC(kernel=self.kernel)
        self.model.fit(X, y)
        return self
        
    def predict(X, y):
        pass

        

In [183]:
model = EmotionRecSVC('rbf')

In [184]:
model.fit_transform(X_train, y_train)

INFO:__main__:Beginning SIFT transformations


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin



class EmotionRecMLP:
    pass

class EmotionRecCNN(nn.Module):
    pass

In [6]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier


X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X


net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X, y)
y_proba = net.predict_proba(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6867       0.6150        0.6808  0.0470
      2        0.6783       0.6600        0.6745  0.0130
      3        0.6617       0.6700        0.6653  0.0120
      4        0.6559       0.6750        0.6557  0.0120
      5        0.6469       0.6900        0.6468  0.0130
      6        0.6382       0.6950        0.6356  0.0120
      7        0.6325       0.7150        0.6233  0.0160
      8        0.6164       0.7150        0.6112  0.0120
      9        0.6258       0.7200        0.6059  0.0140
     10        0.6167       0.7250        0.6005  0.0130


<ipython-input-6-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))
<ipython-input-6-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))
<ipython-input-6-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))
<ipython-input-6-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))
<ipython-input-6-467d7b16a0d1>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  X = F.softmax(self.output(X))
<ipython-input-6-467d7b16a0d1>:27: UserWarning: Implicit dim

In [15]:
import os
from urllib import request
from zipfile import ZipFile

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms

from skorch import NeuralNetClassifier
from skorch.helper import predefined_split

torch.manual_seed(360);

In [8]:
def download_and_extract_data(dataset_dir='datasets'):
    data_zip = os.path.join(dataset_dir, 'hymenoptera_data.zip')
    data_path = os.path.join(dataset_dir, 'hymenoptera_data')
    url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

    if not os.path.exists(data_path):
        if not os.path.exists(data_zip):
            print("Starting to download data...")
            data = request.urlopen(url, timeout=15).read()
            with open(data_zip, 'wb') as f:
                f.write(data)

        print("Starting to extract data...")
        with ZipFile(data_zip, 'r') as zip_f:
            zip_f.extractall(dataset_dir)
        
    print("Data has been downloaded and extracted to {}.".format(dataset_dir))
    
download_and_extract_data()

Data has been downloaded and extracted to datasets.


In [9]:
data_dir = 'datasets/hymenoptera_data'
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

train_ds = datasets.ImageFolder(
    os.path.join(data_dir, 'train'), train_transforms)
val_ds = datasets.ImageFolder(
    os.path.join(data_dir, 'val'), val_transforms)

In [16]:
class PretrainedModel(nn.Module):
    def __init__(self, output_features):
        super().__init__()
        model = models.resnet18(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, output_features)
        self.model = model
        
    def forward(self, x):
        return self.model(x)

In [17]:
from skorch.callbacks import LRScheduler

lrscheduler = LRScheduler(
    policy='StepLR', step_size=7, gamma=0.1)

In [18]:
from skorch.callbacks import Checkpoint

checkpoint = Checkpoint(
    f_params='best_model.pt', monitor='valid_acc_best')



In [19]:
from skorch.callbacks import Freezer

freezer = Freezer(lambda x: not x.startswith('model.fc'))

In [20]:
net = NeuralNetClassifier(
    PretrainedModel, 
    criterion=nn.CrossEntropyLoss,
    lr=0.001,
    batch_size=4,
    max_epochs=25,
    module__output_features=2,
    optimizer=optim.SGD,
    optimizer__momentum=0.9,
    iterator_train__shuffle=True,
    iterator_train__num_workers=4,
    iterator_valid__shuffle=True,
    iterator_valid__num_workers=4,
    train_split=predefined_split(val_ds),
    callbacks=[lrscheduler, checkpoint, freezer],
    device='cuda' # comment to train on cpu
)

c:\users\brenner\documents\python_projects\emotion_recognition\venv\lib\site-packages\skorch\net.py:1536: UserWarning: You set iterator_valid__shuffle=True; this is most likely not what you want because the values returned by predict and predict_proba will be shuffled.
  warnings.warn(


In [23]:
net.fit(train_ds, y=None);

Re-initializing module because the following parameters were re-set: output_features.
Re-initializing optimizer because the following parameters were re-set: momentum.
  epoch    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  ------------  -----------  ------------  ----  ------  ------
      1        0.5739       0.9346        0.2100     +  0.0010  3.2520
      2        0.4608       0.9412        0.2067     +  0.0010  3.1508
      3        0.4630       0.9477        0.2052     +  0.0010  3.2404
      4        0.4561       0.9346        0.2054        0.0010  3.2204
      5        0.4116       0.9542        0.1720     +  0.0010  3.1967
      6        0.4347       0.9412        0.1827        0.0010  3.2181
      7        0.4326       0.9412        0.2000        0.0010  3.2513
      8        0.3945       0.9412        0.2218        0.0001  3.2449
      9        0.3566       0.9477        0.2099        0.0001  3.2322
     10        0.3580       0.9477        0.2020   

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [27]:
device

'cuda'